In [ ]:
### models architectures
##These are our first architectures from EEGdenoisenet paper and code: 
##https://github.com/ncclabsustech/EEGdenoiseNet

import tensorflow as tf
from tensorflow.keras import datasets, layers, models, Input, Sequential

# Author: Haoming Zhang

def fcNN(datanum):
  model = tf.keras.Sequential()
  model.add(Input(shape=(datanum,)))
  model.add(layers.Dense(datanum, activation=tf.nn.relu ))
  model.add(layers.Dropout(0.3))


  model.add(layers.Dense(datanum))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(datanum))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(datanum))
  model.summary()
  return model


def RNN_lstm(datanum):
  model = tf.keras.Sequential()
  model.add(Input(shape=(datanum,1)))
  model.add(layers.LSTM(1,return_sequences = True ))

  model.add(layers.Flatten())

  model.add(layers.Dense(datanum))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(datanum))
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Dense(datanum))
  model.summary()
  return model


def simple_CNN(datanum):
  model = tf.keras.Sequential()

  model.add(layers.Conv1D(64, 3, strides=1, padding='same',input_shape=[ datanum, 1]))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(64, 3, strides=1, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Conv1D(64, 3, strides=1, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  #num4
  model.add(layers.Conv1D(64, 3, strides=1, padding='same'))
  model.add(layers.BatchNormalization())
  model.add(layers.ReLU())
  model.add(layers.Dropout(0.3))

  model.add(layers.Flatten())
  model.add(layers.Dense(datanum))

  model.summary()

  return model


# Resnet Basic Block module。
class Res_BasicBlock(layers.Layer):
  def __init__(self,kernelsize, stride=1):
    super(Res_BasicBlock, self).__init__()
    self.bblock = Sequential([layers.Conv1D(32,kernelsize,strides=stride,padding="same"),
                              layers.BatchNormalization(),
                              layers.ReLU(),
                              layers.Conv1D(16,kernelsize,strides=1,padding="same"),
                              layers.BatchNormalization(),
                              layers.ReLU(),
                              layers.Conv1D(32,kernelsize,strides=1,padding="same"),
                              layers.BatchNormalization(),
                              layers.ReLU()])
                              
    self.jump_layer = lambda x:x


  def call(self, inputs, training=None):

    #Through the convolutional layer
    out = self.bblock(inputs)

    #skip
    identity = self.jump_layer(inputs)

    output = layers.add([out, identity])  #layers下面有一个add，把这2个层添加进来相加。
    
    return output


class BasicBlockall(layers.Layer):
  def __init__(self, stride=1):
    super(BasicBlockall, self).__init__()

    self.bblock3 = Sequential([Res_BasicBlock(3),
                              Res_BasicBlock(3)
                              ])                      
    
    self.bblock5 = Sequential([Res_BasicBlock(5),
                              Res_BasicBlock(5)
                              ])                      

    self.bblock7 = Sequential([Res_BasicBlock(7),
                              Res_BasicBlock(7)
                              ])
                              
    self.downsample = lambda x:x


  def call(self, inputs, training=None):
 
    out3 = self.bblock3(inputs)
    out5 = self.bblock5(inputs)
    out7 = self.bblock7(inputs)

    out = tf.concat( values = [out3,out5,out7] , axis = -1)

    return out


def Complex_CNN(datanum):
  model = Sequential()
  model.add(layers.Conv1D(32 ,5,strides=1,padding="same",input_shape=( datanum, 1)))
  model.add(layers.BatchNormalization())
  model.add( layers.ReLU())

  model.add(BasicBlockall())

  model.add(layers.Conv1D(32 ,1,strides=1,padding="same"))
  model.add(layers.BatchNormalization())
  model.add( layers.ReLU())

  model.add(layers.Flatten())
  model.add(layers.Dense(datanum))

  model.summary()
  
  return model

In [ ]:
####this is the last architecture, called LU-net architecture
##we can find the paper here: https://ieeexplore.ieee.org/abstract/document/10173517

#LU-Net architecture
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, ReLU, Bidirectional, LSTM, UpSampling1D, Concatenate, Input, Conv2DTranspose

def encoder_decoder(datanum):
    # Encoder
    input_layer = Input(shape=(datanum,1))
    x = input_layer
    encoder_outputs = []
    lstm_outputs =[]
    filters_encoder = [16, 32, 32, 64, 64]
    filters_lstm = [8, 16,16,32,32] 
    strides_encoder = [1,2,2,2,2]
    for i in range(1, 6):
        if i > 1:
            x = encoder_outputs[-1]  # Pass output from previous encoder as input
        x = Conv1D(filters=filters_encoder[i-1], kernel_size=31, strides=strides_encoder[i-1], padding='same')(x)
        x = ReLU()(x)
        encoder_outputs.append(x)
        x = Bidirectional(LSTM(units=filters_lstm[i-1],return_sequences=True))(encoder_outputs[-1])
        lstm_outputs.append(x)

    
    # Bottleneck
    bottleneck_output = Conv1D(filters=128, kernel_size=31, strides=2, padding='same')(encoder_outputs[-1])
    
    # Decoder
    decoder_outputs = []
    filters_decoder = [32,32,32,64, 64]               
    for i in range(5, 0, -1):
        if i < 5:
            x = Concatenate()([decoder_outputs[-1], lstm_outputs[i]])  # Concatenate with LSTM output
        else:
            x = bottleneck_output
        x = Conv1D(filters=filters_decoder[i-1], kernel_size=31,strides= 1, padding='same')(x)
        x = ReLU()(x)
        x = UpSampling1D()(x)
        decoder_outputs.append(x)
    
    # Final convolution layer
    x = Concatenate()([decoder_outputs[-1], lstm_outputs[0]])
    output = Conv1D(filters=1, kernel_size=31 ,strides= 1, padding='same')(x)
    output = tf.keras.layers.Reshape((datanum,))(output)
    
    # Define model
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.summary()
    return model

In [ ]:
#LU-Net architecture 3 layers
import tensorflow as tf
from tensorflow.keras.layers import Conv1D, ReLU, Bidirectional, LSTM, UpSampling1D, Concatenate, Input, Conv2DTranspose

def encoder_decoder(datanum):
    # Encoder
    input_layer = Input(shape=(datanum,1))
    x = input_layer
    encoder_outputs = []
    lstm_outputs =[]
    filters_encoder = [16, 32, 32]
    filters_lstm = [8, 16,16] 
    strides_encoder = [1,2,2]
    for i in range(1, 4):
        if i > 1:
            x = encoder_outputs[-1]  # Pass output from previous encoder as input
        x = Conv1D(filters=filters_encoder[i-1], kernel_size=31, strides=strides_encoder[i-1], padding='same')(x)
        x = ReLU()(x)
        encoder_outputs.append(x)
        x = Bidirectional(LSTM(units=filters_lstm[i-1],return_sequences=True))(encoder_outputs[-1])
        lstm_outputs.append(x)

    
    # Bottleneck
    bottleneck_output = Conv1D(filters=128, kernel_size=31, strides=2, padding='same')(encoder_outputs[-1])
    
    # Decoder
    decoder_outputs = []
    filters_decoder = [32,32,32]               
    for i in range(3, 0, -1):
        if i < 3:
            x = Concatenate()([decoder_outputs[-1], lstm_outputs[i]])  # Concatenate with LSTM output
        else:
            x = bottleneck_output
        x = Conv1D(filters=filters_decoder[i-1], kernel_size=31,strides= 1, padding='same')(x)
        x = ReLU()(x)
        x = UpSampling1D()(x)
        decoder_outputs.append(x)
    
    # Final convolution layer
    x = Concatenate()([decoder_outputs[-1], lstm_outputs[0]])
    output = Conv1D(filters=1, kernel_size=31 ,strides= 1, padding='same')(x)
    output = tf.keras.layers.Reshape((datanum,))(output)
    
    # Define model
    model = tf.keras.Model(inputs=input_layer, outputs=output)
    model.summary()
    return model

In [ ]:
###########a code for preparing the data

import sklearn.model_selection as ms
import numpy as np
import scipy.io as sio
import math
# Author: Haoming Zhang
#The code here not only include data importing, but also data standardization and the generation of analog noise signals

def get_rms(records):
   
    return math.sqrt(sum([x ** 2 for x in records]) / len(records))


def random_signal(signal,combin_num):
    # Random disturb and augment signal
    random_result=[]

    for i in range(combin_num):
        random_num = np.random.permutation(signal.shape[0])
        shuffled_dataset = signal[random_num, :]
        shuffled_dataset = shuffled_dataset.reshape(signal.shape[0],signal.shape[1])
        random_result.append(shuffled_dataset)
    
    random_result  = np.array(random_result)

    return  random_result



def prepare_data(EEG_all, EMG_all , EOG_all, combin_num, train_per, noise_type):
    # Here we use eeg and noise signal to generate scale transed training, validation, test signal
    EEG_all_random = np.squeeze(random_signal(signal = EEG_all, combin_num = 1))
    noise_all_random = np.squeeze(random_signal(signal = EMG_all, combin_num = 1))
    EOG_all_random = np.squeeze(random_signal(signal = EOG_all, combin_num = 1))

    if noise_type == 'EMG':  # Training set will Reuse some of the EEG signal to much the number of EMG
        reuse_num = noise_all_random.shape[0] - EEG_all_random.shape[0]
        EEG_reuse = EEG_all_random[0 : reuse_num, :]
        EEG_all_random = np.vstack([EEG_reuse, EEG_all_random])
        print('EEG segments after reuse: ',EEG_all_random.shape[0])

    elif noise_type == 'EOG':  # We will drop some of the EEG signal to much the number of EMG
        EEG_all_random = EEG_all_random[0:noise_all_random.shape[0]]
        print('EEG segments after drop: ',EEG_all_random.shape[0])
    
    # get the 
    timepoint = noise_all_random.shape[1]
    train_num = round(train_per * EEG_all_random.shape[0]) # the number of segmentations used in training process
    validation_num = round((EEG_all_random.shape[0] - train_num) / 2) # the number of segmentations used in validation process
    #test_num = EEG_all_random.shape[0] - train_num - validation_num  # Rest are the number of segmentations used in test process

    train_eeg = EEG_all_random[0 : train_num, :]
    validation_eeg = EEG_all_random[train_num : train_num + validation_num, :]
    test_eeg = EEG_all_random[train_num + validation_num : EEG_all_random.shape[0], :]

    train_noise = noise_all_random[0 : train_num, :]
    validation_noise = noise_all_random[train_num : train_num + validation_num,:]
    test_noise = noise_all_random[train_num + validation_num : noise_all_random.shape[0], :]
    
    
    EEG_train = random_signal(signal = train_eeg, combin_num = combin_num).reshape(combin_num * train_eeg.shape[0], timepoint)
    #####using amp wraping
#     NOISE_train = magnitude_warping(train_noise)
    #########
    NOISE_train = random_signal(signal = train_noise, combin_num = combin_num).reshape(combin_num * train_noise.shape[0], timepoint)
    ###########################################
    #print(EEG_train.shape)
    #print(NOISE_train.shape)
    
    #################################  simulate noise signal of training set  ##############################

    #create random number between -10dB ~ 2dB
    SNR_train_dB = np.random.uniform(-7, 4, (EEG_train.shape[0])) ##### 4 for EMG
    print(SNR_train_dB.shape)
    SNR_train = 10 ** (0.1 * (SNR_train_dB))

    # combin eeg and noise for training set 
    noiseEEG_train=[]
    NOISE_train_adjust=[]
    print('EEG train: ',EEG_train.shape)
    for i in range (EEG_train.shape[0]):
        eeg=EEG_train[i].reshape(EEG_train.shape[1])
        noise=NOISE_train[i].reshape(NOISE_train.shape[1])

        coe=get_rms(eeg)/(get_rms(noise)*SNR_train[i])
        noise = noise*coe
        neeg = noise+eeg

        NOISE_train_adjust.append(noise)
        noiseEEG_train.append(neeg)

    
    noiseEEG_train=np.array(noiseEEG_train)
    NOISE_train_adjust=np.array(NOISE_train_adjust)
    print('noiseEEG_train',noiseEEG_train.shape)
    

    # variance for noisy EEG
#     EEG_train_end_standard = []
#     noiseEEG_train_end_standard = []

#     for i in range(noiseEEG_train.shape[0]):
#         # Each epochs divided by the standard deviation
#         eeg_train_all_std = EEG_train[i] / np.std(noiseEEG_train[i])
#         EEG_train_end_standard.append(eeg_train_all_std)

#         noiseeeg_train_end_standard = noiseEEG_train[i] / np.std(noiseEEG_train[i])
#         noiseEEG_train_end_standard.append(noiseeeg_train_end_standard)

#     noiseEEG_train_end_standard = np.array(noiseEEG_train_end_standard)
#     EEG_train_end_standard = np.array(EEG_train_end_standard)
#     print('training data prepared', noiseEEG_train_end_standard.shape, EEG_train_end_standard.shape )

    #################################  simulate noise signal of validation  ##############################

#     SNR_val_dB = np.linspace(-7.0, 4.0, num=(10))
#     np.random.shuffle(SNR_val_dB)
#     SNR_val = 10 ** (0.1 * (SNR_val_dB))

    eeg_val = np.array(validation_eeg)
    noise_val = np.array(validation_noise)
    
    noise_val = random_signal(signal = noise_val, combin_num = combin_num).reshape(combin_num * noise_val.shape[0], timepoint)
    eeg_val = random_signal(signal = eeg_val, combin_num = combin_num).reshape(combin_num * eeg_val.shape[0], timepoint)
    
    SNR_val_dB = np.random.uniform(-7, 4, (eeg_val.shape[0]))
    SNR_val = 10 ** (0.1 * (SNR_val_dB))
    # combin eeg and noise for test set 
    EEG_val = []
    noise_EEG_val = []  
        
    noise_eeg_val = []
    for j in range(eeg_val.shape[0]):
        eeg = eeg_val[j]
        noise = noise_val[j]
            
        coe = get_rms(eeg) / (get_rms(noise) * SNR_val[j])
        noise = noise * coe
        neeg = noise + eeg
            
        noise_eeg_val.append(neeg)
        
    #################################  simulate noise signal of test  ##############################

    SNR_test_dB = np.linspace(-7.0, 2.0, num=(10))
    SNR_test = 10 ** (0.1 * (SNR_test_dB))

    eeg_test = np.array(test_eeg)
    noise_test = np.array(test_noise)
    
    # combin eeg and noise for test set 
    EEG_test = []
    noise_EEG_test = []
    for i in range(10): 
        
        noise_eeg_test = []
        for j in range(eeg_test.shape[0]):
            eeg = eeg_test[j]
            noise = noise_test[j]
            
            coe = get_rms(eeg) / (get_rms(noise) * SNR_test[i])
            noise = noise* coe
            neeg = noise + eeg
            
            noise_eeg_test.append(neeg)
        
    ##########EOG splitting
    
    EOG_all_random = np.squeeze(random_signal(signal = EOG_all, combin_num = 1))
    reuse_num_eog = noise_all_random.shape[0] - EOG_all_random.shape[0]
    EOG_reuse = EOG_all_random[0 : reuse_num_eog, :] 
    EOG = np.vstack([EOG_reuse, EOG_all_random])
    print(EOG.shape) ####5598 1024

    
    print('EOG segments: ',EOG.shape[0])
    train_eog = EOG[0 : train_num, :]
    validation_eog = EOG[train_num : train_num + validation_num,:]
    test_eog = EOG[train_num + validation_num : noise_all_random.shape[0], :]
    
    ################increasing EOG rows by shifting augmentation
#     EOG_train = geometric_augmentation(train_eog)
    #####################randomly increasing EOG rows##############
    EOG_train = random_signal(signal = train_eog, combin_num = combin_num).reshape(combin_num * train_eog.shape[0], timepoint)
    ##########################
    print('EOG train: ',EOG_train.shape[0])
    
    
    #########for train EOG
    #create random number between -10dB ~ 2dB
    SNR_train_dB = np.random.uniform(-7, 2, (EEG_train.shape[0])) 
    print(SNR_train_dB.shape)
    SNR_train = 10 ** (0.1 * (SNR_train_dB))

    # combin eeg and noise for training set 
    noiseEEG_train_final=[]
    EOG_train_adjust=[]
    for i in range (EEG_train.shape[0]):
        eeg=EEG_train[i].reshape(EEG_train.shape[1])
        eog=EOG_train[i].reshape(EOG_train.shape[1])

        coe_eog=get_rms(eeg)/(get_rms(eog)*SNR_train[i])
        eog = eog*coe_eog
        cont = eog+noiseEEG_train[i]

        EOG_train_adjust.append(eog)
        noiseEEG_train_final.append(cont)

    noiseEEG_train_final=np.array(noiseEEG_train_final)
    EOG_train_adjust=np.array(EOG_train_adjust)
    print('EEG train: ',EEG_train.shape)
    print('contam final',noiseEEG_train_final.shape)
           
    #############for val EOG
#     SNR_val_dB = np.linspace(-7.0, 2.0, num=(10))
#     SNR_val = 10 ** (0.1 * (SNR_val_dB))

    eog_val = random_signal(signal = validation_eog, combin_num = combin_num).reshape(combin_num * validation_eog.shape[0], timepoint)
    SNR_val_dB = np.random.uniform(-7, 2, (eog_val.shape[0]))
    SNR_val = 10 ** (0.1 * (SNR_val_dB))
    
    # combin eeg and noise for test set 
    EEG_val = []
    noise_EEG_val_final = []

    noise_eeg_val_final = []
    for j in range(eeg_val.shape[0]):
        eeg = eeg_val[j]
        eog = eog_val[j]
            
        coe_eog = get_rms(eeg) / (get_rms(eog) * SNR_val[j])
        eog = eog * coe_eog
        cont = eog + noise_eeg_val[j]   ####EMG EOG
            
        noise_eeg_val_final.append(cont)
        
    EEG_val.extend(eeg_val)
    noise_EEG_val_final.extend(noise_eeg_val_final)


    EEG_val = np.array(EEG_val)
    noise_EEG_val_final = np.array(noise_EEG_val_final)
           
    ################test EOG
    SNR_test_dB = np.linspace(-7.0, 2.0, num=(10))
    SNR_test = 10 ** (0.1 * (SNR_test_dB))

    eog_test = np.array(test_eog)
    
    # combin eeg and noise for test set 
    EEG_test = []
    noise_EEG_test_final = []
    for i in range(10): 
        
        noise_eeg_test_final = []
        for j in range(eeg_test.shape[0]):
            eeg = eeg_test[j]
            eog = eog_test[j]
            
            coe_eog = get_rms(eeg) / (get_rms(eog) * SNR_test[i])
            eog = eog* coe_eog
            cont = eog + noise_eeg_test[j]   #####EOG EMG
            
            noise_eeg_test_final.append(cont)
        
        EEG_test.extend(eeg_test)
        noise_EEG_test_final.extend(noise_eeg_test_final)

    EEG_test = np.array(EEG_test)
    noise_EEG_test_final = np.array(noise_EEG_test_final)
    

           
    ##############standardization train data
           
    EEG_train_end_standard = []
    noiseEEG_train_end_standard = []

    for i in range(noiseEEG_train_final.shape[0]):
        # Each epochs divided by the standard deviation
        eeg_train_all_std = EEG_train[i] / np.std(noiseEEG_train_final[i])
        EEG_train_end_standard.append(eeg_train_all_std)

        noiseeeg_train_end_standard = noiseEEG_train_final[i] / np.std(noiseEEG_train_final[i])
        noiseEEG_train_end_standard.append(noiseeeg_train_end_standard)

    noiseEEG_train_end_standard = np.array(noiseEEG_train_end_standard)
    EEG_train_end_standard = np.array(EEG_train_end_standard)
    print('training data prepared', noiseEEG_train_end_standard.shape, EEG_train_end_standard.shape )

           
    ##############standardization validation data
    EEG_val_end_standard = []
    noiseEEG_val_end_standard = []

    for i in range(noise_EEG_val_final.shape[0]):
        
        # store std value to restore EEG signal
        std_value = np.std(noise_EEG_val_final[i])
        #std_VALUE.append(std_value)

        # Each epochs of eeg and neeg was divide by the standard deviation
        eeg_val_all_std = EEG_val[i] / std_value
        EEG_val_end_standard.append(eeg_val_all_std)

        noiseeeg_val_end_standard = noise_EEG_val_final[i] / std_value
        noiseEEG_val_end_standard.append(noiseeeg_val_end_standard)

    #std_VALUE = np.array(std_VALUE)
    noiseEEG_val_end_standard = np.array(noiseEEG_val_end_standard)
    EEG_val_end_standard = np.array(EEG_val_end_standard)
    print('validation data prepared, validation data shape: ', noiseEEG_val_end_standard.shape, EEG_val_end_standard.shape)
    
           
           
    ##############standardization test data
    EEG_test_end_standard = []
    noiseEEG_test_end_standard = []
    std_VALUE = []
    for i in range(noise_EEG_test_final.shape[0]):
        
        # store std value to restore EEG signal
        std_value = np.std(noise_EEG_test_final[i])
        std_VALUE.append(std_value)

        # Each epochs of eeg and neeg was divide by the standard deviation
        eeg_test_all_std = EEG_test[i] / std_value
        EEG_test_end_standard.append(eeg_test_all_std)

        noiseeeg_test_end_standard = noise_EEG_test_final[i] / std_value
        noiseEEG_test_end_standard.append(noiseeeg_test_end_standard)

    std_VALUE = np.array(std_VALUE)
    noiseEEG_test_end_standard = np.array(noiseEEG_test_end_standard)
    EEG_test_end_standard = np.array(EEG_test_end_standard)
    print('test data prepared, test data shape: ', noiseEEG_test_end_standard.shape, EEG_test_end_standard.shape)       
           
           
    return noiseEEG_train_end_standard, EEG_train_end_standard, noiseEEG_val_end_standard, EEG_val_end_standard, noiseEEG_test_end_standard, EEG_test_end_standard, std_VALUE
  

In [ ]:
##functions for calculating the loss
##rrmse is the most used by us

import tensorflow as tf

################################# loss functions ##########################################################

def denoise_loss_mse(denoise, clean):      
  loss = tf.losses.mean_squared_error(denoise, clean)
  return tf.reduce_mean(loss)

def denoise_loss_rmse(denoise, clean):      #tmse
  loss = tf.losses.mean_squared_error(denoise, clean)
  #loss2 = tf.losses.mean_squared_error(noise, clean)
  return tf.math.sqrt(tf.reduce_mean(loss))

def denoise_loss_rrmset(denoise, clean):      #tmse
  rmse1 = denoise_loss_rmse(denoise, clean)
  rmse2 = denoise_loss_rmse(clean, tf.zeros(clean.shape[0], tf.float64))
  #loss2 = tf.losses.mean_squared_error(noise, clean)
  return rmse1/rmse2

# **for normal run:**

In [ ]:
###train code

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import os  
import math
import datetime

# Author: Haoming Zhang
# Here is the part of denoiseNet training process
    
@tf.function
def train_step(model, noiseEEG_batch, EEG_batch, optimizer , denoise_network, batch_size, datanum):

    #本次训练参数初始化  parameter initialization in one step

    mse_grads = 0
    m_loss = 0
      
 
    with tf.GradientTape() as loss_tape:
    
        M_loss =  0
        for x in range(noiseEEG_batch.shape[0]):
    
            noiseeeg_batch,eeg_batch =  noiseEEG_batch[x] , EEG_batch[x]

            if denoise_network == 'fcNN':
                noiseeeg_batch = tf.reshape(noiseeeg_batch, [1,datanum])
            else:
                noiseeeg_batch = tf.reshape(noiseeeg_batch, [1,datanum,1])

            eeg_batch=tf.reshape(eeg_batch, [1,datanum,1])
            denoiseoutput = model(noiseeeg_batch)
            denoiseoutput = tf.reshape(denoiseoutput, [1,datanum,1])                          

            m_loss = denoise_loss_mse(denoiseoutput,eeg_batch)   
            M_loss += m_loss

    
        M_loss = M_loss / float(noiseEEG_batch.shape[0]) 
        
        # calculate gradient
        mse_grads = loss_tape.gradient(M_loss, model.trainable_variables)
        #bp
        optimizer.apply_gradients(zip(mse_grads, model.trainable_variables))

    return  M_loss,  mse_grads[0]  #每一条EEG的loss从此输出

def test_step(model, noiseEEG_test, EEG_test):

  denoiseoutput_test = model(noiseEEG_test)
  loss = denoise_loss_mse(EEG_test, denoiseoutput_test)
  #loss_rrmset = denoise_loss_rrmset(denoiseoutput_test, EEG_test)

  return denoiseoutput_test, loss#, loss_rrmset


def train(model, noiseEEG,EEG, noiseEEG_val, EEG_val, epochs, batch_size,optimizer, denoise_network, result_location, foldername, train_num,prev_history,history):
    if prev_history ==False:
        # setup history variables and save history in a npy film
        history = {}
        history['grads'], history['loss']= {}, {}
        train_mse_history, val_mse_history = [],[]
        mse_grads_history = []
        val_mse_min = 100.0      # any number bigger than 1
    else:
        history=history
        train_mse_history, val_mse_history = history['loss']['train_mse'], history['loss']['val_mse']
        mse_grads_history = history['grads']['mse'] 
        hlv=history['loss']['val_mse']
        val_mse_min = float(hlv[-1]) # any number bigger than 1
    
        
    # save history to tensorboard
    # current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    train_log_dir = result_location +'/'+foldername +'/'+ train_num + '/train'
    val_log_dir = result_location +'/'+foldername +'/'+ train_num + '/test'
    train_summary_writer = tf.summary.create_file_writer(train_log_dir)
    val_summary_writer = tf.summary.create_file_writer(val_log_dir)

    batch_num = math.ceil(noiseEEG.shape[0]/batch_size)
    
    datanum = noiseEEG.shape[1]
    for epoch in range(epochs):
        start = time.time()

        # initialize  loss value for every epoch
        mse_grads , train_mse = 0, 0

        with tqdm(total=batch_num, position=0, leave=True) as pbar:
    
            for n_batch in range(batch_num):

                #
                if n_batch == batch_num:
                    noiseEEG_batch,EEG_batch =  noiseEEG[batch_size*n_batch :] , EEG[batch_size*n_batch :]
                else:
                    noiseEEG_batch,EEG_batch =  noiseEEG[batch_size*n_batch : batch_size*(n_batch+1)] , EEG[batch_size*n_batch : batch_size*(n_batch+1)]

                mse_loss_batch, mse_grads_batch = train_step(model, noiseEEG_batch,EEG_batch, optimizer, denoise_network, batch_size , datanum)

                # convert variables to usable format
                mse_grads_batch = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(mse_grads_batch)))).numpy()
                mse_loss_batch = tf.reduce_mean(mse_loss_batch).numpy()

                # store history 
                train_mse += mse_loss_batch/float(batch_num)
                mse_grads += mse_grads_batch/float(batch_num)

                pbar.update()
            pbar.close()

        # store train history 
        mse_grads_history.append(mse_grads)
        train_mse_history.append(train_mse)

        with train_summary_writer.as_default():
            tf.summary.scalar('loss', train_mse, step=epoch)


        # calculate mse loss for validation set
        #denoiseoutput, val_mse, loss_rrmset = test_step(model, noiseEEG_val, EEG_val)
        denoiseoutput, val_mse = test_step(model, noiseEEG_val, EEG_val)

        #store validation history
        val_mse_history.append(val_mse) 

        with val_summary_writer.as_default():   # record validation loss to tensorboard
            tf.summary.scalar('loss', val_mse, step=epoch)

        if epoch>epochs*0.2 and float(val_mse) < val_mse_min:  # if epoch_number > 0.8*all_epoch_number begin to save the best model  ## for SCNN or CCNN in EMG we should save the first or second model. 
            print('yes,smaller ', float(val_mse) ,val_mse_min)  
            val_mse_min = float(val_mse)
            saved_model = model
            
            # save dictionary to person_data.pkl file
            with open('/kaggle/working/forpretrained/history.pkl', 'wb') as fp:
                pickle.dump(history, fp)
            #save model
            model.save("/kaggle/working/forpretrained/my_model.keras",saved_model)

            path = os.path.join(result_location, foldername, train_num, "denoise_model")
            tf.keras.models.save_model(model, path)
            print('Best model has been saved')

        print ('Epoch #: {}/{}, Time taken: {} secs,\n Grads: mse= {},\n Losses: train_mse= {}, val_mse={}'\
                     .format(epoch+1,epochs,time.time()-start , mse_grads,  train_mse, val_mse))

            
    #Generate after the final epoch
    clear_output(wait=True)

    #save history to dict
    history['grads']['mse'] = mse_grads_history
    history['loss']['train_mse'], history['loss']['val_mse']  = train_mse_history, val_mse_history
        
    return saved_model, history  

In [ ]:
#### save

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import os  
import math


def save_eeg(saved_model, result_location, foldername, save_train, save_vali, save_test, 
            noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, noiseEEG_test, EEG_test, train_num):

    if save_train == True:
        # generate every signal in training set
        Denoiseoutput_train, train_mse = test_step(saved_model, noiseEEG_train, EEG_train)    

        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/' +"nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"   )
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' + "noiseinput_train.npy", noiseEEG_train)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' +  "Denoiseoutput_train.npy", Denoiseoutput_train)               #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' +  "EEG_train.npy", EEG_train)

    if save_vali == True:
        # generate every signal in test set
        Denoiseoutput_val, val_mse = test_step(saved_model, noiseEEG_val, EEG_val)        
            
        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output")    
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "noiseinput_val.npy", noiseEEG_val)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' +  "Denoiseoutput_val.npy", Denoiseoutput_val)                      #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "EEG_val.npy", EEG_val)
        
    if save_test == True:
        # generate every signal in test set

        Denoiseoutput_test, test_mse = test_step(saved_model, noiseEEG_test, EEG_test)


        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/' + "nn_output")    
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "noiseinput_test.npy", noiseEEG_test)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' +  "Denoiseoutput_test.npy", Denoiseoutput_test)                      #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "EEG_test.npy", EEG_test)

In [ ]:
### main

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import sys
import os
#saving history
import pickle
#sys.path.append('../')
import copy

# EEGdenoiseNet V2
# Author: Haoming Zhang 
# Here is the main part of the denoising neurl network, We can adjust all the parameter in the user-defined area.
#####################################################自定义 user-defined ########################################################

epochs =20 # training epoch
batch_size  = 40    # training batch size
combin_num = 10    # combin EEG and noise ? times
denoise_network = 'Complex_CNN'    # fcNN & Simple_CNN & Complex_CNN & RNN_lstm  & Novel_CNN 
noise_type = 'EMG'


result_location = r'/kaggle/working/'     #  Where to export network results   ############ change it to your own location #########
foldername = 'EMG_unet112dense_10_rmsp_test'            # the name of the target folder (should be change when we want to train a new network)
os.environ['CUDA_VISIBLE_DEVICES']='0'
save_train = False
save_vali = False
save_test = True


################################################## optimizer adjust parameter  ####################################################
rmsp=tf.optimizers.RMSprop(learning_rate=0.00005, rho=0.9)
adam=tf.optimizers.Adam(learning_rate=0.00005, beta_1=0.5, beta_2=0.9, epsilon=1e-08)
# sgd=tf.optimizers.legacy.SGD(learning_rate=0.0002, momentum=0.9, decay=0.0, nesterov=False)

optimizer = rmsp

if noise_type == 'EOG':
  datanum = 1024  ##########changed
elif noise_type == 'EMG':
  datanum =1024


# We have reserved an example of importing an existing network
'''
path = os.path.join(result_location, foldername, "denoised_model")
denoiseNN = tf.keras.models.load_model(path)
'''
#################################################### 数据输入 Import data #####################################################

file_location = '/kaggle/input/eeg-data/'                    ############ change it to your own location #########
if noise_type == 'EOG':
  EEG_all = np.load( file_location + 'EEG_all_epochs_512hz.npy')                              
  noise_all = np.load( file_location + 'EOG_all_epochs.npy') 
elif noise_type == 'EMG':
  EEG_all = np.load( file_location + 'EEG_all_epochs_512hz.npy')                              
  noise_all = np.load( file_location + 'EMG_all_epochs_512hz.npy')

import scipy.io
EOG_512Hz=scipy.io.loadmat("/kaggle/input/eog-all-epochs-512hz/eog_512Hz (1).mat")
EOG_data_512Hz=EOG_512Hz['eog_512Hz']

############################################################# Running #############################################################
#for i in range(10):
i = 1     # We run each NN for 10 times to increase  the  statistical  power  of  our  results

prev_history=False
history={}
if denoise_network == 'fcNN':
    model = fcNN(datanum)

elif denoise_network == 'Simple_CNN':
  model = simple_CNN(datanum)

elif denoise_network == 'Complex_CNN':
  model = Complex_CNN(datanum)

elif denoise_network == 'RNN_lstm':
  model = RNN_lstm(datanum)

elif denoise_network == 'Novel_CNN':
  model = Novel_CNN(datanum)


else: 
    print('You are editing the same model')
    model =tf.keras.saving.load_model("/kaggle/working/forpretrained/my_model.keras")
    #load
    file_to_read = open("/kaggle/working/forpretrained/history.pkl", "rb")
    history = pickle.load(file_to_read)
    print(history)
    prev_history=True
    noiseEEG_train=np.load("/kaggle/working/forpretrained/noiseEEG_train.npy")
    EEG_train=np.load("/kaggle/working/forpretrained/EEG_train.npy")
    noiseEEG_val=np.load("/kaggle/working/forpretrained/noiseEEG_val.npy")
    EEG_val=np.load("/kaggle/working/forpretrained/EEG_val.npy")
    noiseEEG_test=np.load("/kaggle/working/forpretrained/noiseEEG_test.npy")
    EEG_test=np.load("/kaggle/working/forpretrained/EEG_test.npy")
if prev_history==False:
    noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, noiseEEG_test, EEG_test, test_std_VALUE = prepare_data(EEG_all = EEG_all, EMG_all = noise_all,EOG_all=EOG_data_512Hz, combin_num = 10, train_per = 0.8, noise_type = noise_type)
    if not os.path.exists("/kaggle/working/forpretrained"):
                os.makedirs("/kaggle/working/forpretrained")
    np.save("/kaggle/working/forpretrained/noiseEEG_train.npy",noiseEEG_train)
    np.save("/kaggle/working/forpretrained/EEG_train.npy",EEG_train)
    np.save("/kaggle/working/forpretrained/noiseEEG_val.npy",noiseEEG_val)
    np.save("/kaggle/working/forpretrained/EEG_val.npy",EEG_val)
    np.save("/kaggle/working/forpretrained/noiseEEG_test.npy",noiseEEG_test)
    np.save("/kaggle/working/forpretrained/EEG_test.npy",EEG_test)
    ###may save test_std_VALUE



saved_model, history = train(model, noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, 
                      epochs, batch_size,optimizer, denoise_network, 
                      result_location, foldername , train_num = str(i),prev_history=prev_history,history=history)

#denoised_test, test_mse = test_step(saved_model, noiseEEG_test, EEG_test)

# save signal
save_eeg(saved_model, result_location, foldername, save_train, save_vali, save_test, 
                    noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, noiseEEG_test, EEG_test, 
                    train_num = str(i))
np.save(result_location +'/'+ foldername + '/'+ str(i)  +'/'+ "nn_output" + '/'+ 'loss_history.npy', history)

#save model
# path = os.path.join(result_location, foldername, str(i+1), "denoise_model")
# tf.keras.models.save_model(saved_model, path)
#save
print(history)
# save dictionary to person_data.pkl file
with open('/kaggle/working/forpretrained/history.pkl', 'wb') as fp:
    pickle.dump(history, fp)
    print('dictionary saved successfully to file')
    
#save model , history, data:
model.save("/kaggle/working/forpretrained/my_model.keras",saved_model)

# **for running optuna ***

In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import os  
import math
import datetime

# Author: Haoming Zhang
# Here is the part of denoiseNet training process
    

def test_step(model, noiseEEG_test, EEG_test):

  denoiseoutput_test = model(noiseEEG_test)
  loss = denoise_loss_mse(EEG_test, denoiseoutput_test)
  #loss_rrmset = denoise_loss_rrmset(denoiseoutput_test, EEG_test)

  return denoiseoutput_test, loss#, loss_rrmset


In [ ]:
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import os  
import math


def save_eeg(saved_model, result_location, foldername, save_train, save_vali, save_test, 
            noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, noiseEEG_test, EEG_test, train_num):

    if save_train == True:
        # generate every signal in training set
        Denoiseoutput_train, train_mse = test_step(saved_model, noiseEEG_train, EEG_train)    

        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/' +"nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"   )
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' + "noiseinput_train.npy", noiseEEG_train)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' +  "Denoiseoutput_train.npy", Denoiseoutput_train)               #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" + '/' +  "EEG_train.npy", EEG_train)

    if save_vali == True:
        # generate every signal in test set
        Denoiseoutput_val, val_mse = test_step(saved_model, noiseEEG_val, EEG_val)        
            
        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output")    
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "noiseinput_val.npy", noiseEEG_val)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' +  "Denoiseoutput_val.npy", Denoiseoutput_val)                      #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "EEG_val.npy", EEG_val)
        
    if save_test == True:
        # generate every signal in test set

        Denoiseoutput_test, test_mse = test_step(saved_model, noiseEEG_test, EEG_test)


        if not os.path.exists(result_location +'/'+  foldername + '/' +  train_num + '/'+ "nn_output"):
            os.makedirs(result_location +'/'+  foldername + '/' +  train_num + '/' + "nn_output")    
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "noiseinput_test.npy", noiseEEG_test)
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' +  "Denoiseoutput_test.npy", Denoiseoutput_test)                      #######################   地址要改！！！！！！！！
        np.save(result_location +'/'+  foldername + '/' + train_num + '/' + "nn_output" +'/' + "EEG_test.npy", EEG_test)
        


In [ ]:
!pip install optuna

In [ ]:
###for hyperparameter optimization
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt
import numpy as np
import time
from functools import partial
from tqdm import tqdm
from IPython.display import clear_output 
import sys
import os
#saving history
import pickle
#sys.path.append('../')
import copy

import tensorflow as tf
import torch
import optuna
import logging
import sys
import gc

#################################################### 数据输入 Import data #####################################################

noise_type = 'EMG'
file_location = '/kaggle/input/eeg-data/'     ########## change it to your own location #########
if noise_type == 'EOG':
    EEG_all = np.load( file_location + 'EEG_all_epochs_512hz.npy')                              
    noise_all = np.load( file_location + 'EOG_all_epochs.npy') 
elif noise_type == 'EMG':
    EEG_all = np.load( file_location + 'EEG_all_epochs_512hz.npy')                              
    noise_all = np.load( file_location + 'EMG_all_epochs_512hz.npy')

import scipy.io 
EOG_512Hz=scipy.io.loadmat("/kaggle/input/eog-all-epochs-512hz/eog_512Hz (1).mat")
EOG_data_512Hz=EOG_512Hz['eog_512Hz']

noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, noiseEEG_test, EEG_test, test_std_VALUE = prepare_data(EEG_all = EEG_all, EMG_all = noise_all,EOG_all=EOG_data_512Hz, combin_num = 1, train_per = 0.8, noise_type = 'EMG')
if not os.path.exists("/kaggle/working/forpretrained"):
            os.makedirs("/kaggle/working/forpretrained")
np.save("/kaggle/working/forpretrained/noiseEEG_train.npy",noiseEEG_train)
np.save("/kaggle/working/forpretrained/EEG_train.npy",EEG_train)
np.save("/kaggle/working/forpretrained/noiseEEG_val.npy",noiseEEG_val)
np.save("/kaggle/working/forpretrained/EEG_val.npy",EEG_val)

def objective(trial):
    print('trial number: ',trial.number)
    # Save the current trial number
    trial_num_file = "/kaggle/working/trial_num.pkl"
    with open(trial_num_file, "wb") as f:
        pickle.dump(trial.number, f)
        
    epochs =3 # training epoch
    batch_size  = 40    # training batch size
    denoise_network = 'encoder_decoder'    # fcNN & Simple_CNN & Complex_CNN & RNN_lstm  & Novel_CNN 
    noise_type = 'EMG'


    result_location = r'/kaggle/working/'     #  Where to export network results   ############ change it to your own location #########
    foldername = 'EMG_unet112dense_10_rmsp_test'            # the name of the target folder (should be change when we want to train a new network)
    os.environ['CUDA_VISIBLE_DEVICES']='0'
    save_train = False
    save_vali = False
    save_test = True


    if noise_type == 'EOG':
        datanum = 1024  ##########changed
    elif noise_type == 'EMG':
        datanum =1024


    # We have reserved an example of importing an existing network
    '''
    path = os.path.join(result_location, foldername, "denoised_model")
    denoiseNN = tf.keras.models.load_model(path)
    '''

############################################################# Running #############################################################
    i = 1     # We run each NN for 10 times to increase  the  statistical  power  of  our  results

    prev_history=False
    history={}
    if denoise_network == 'fcNN':
        model = fcNN(datanum)

    elif denoise_network == 'Simple_CNN':
      model = simple_CNN(datanum)

    elif denoise_network == 'Complex_CNN':
      model = Complex_CNN(datanum)

    elif denoise_network == 'RNN_lstm':
      model = RNN_lstm(datanum)

    elif denoise_network == 'Novel_CNN':  
      model = Novel_CNN(datanum)

    elif denoise_network == 'Hybrid':  
      model = Hybrid(datanum)
    elif denoise_network == 'encoder_decoder':  
      model = encoder_decoder(datanum)

    noiseEEG_train=np.load("/kaggle/working/forpretrained/noiseEEG_train.npy")
    EEG_train=np.load("/kaggle/working/forpretrained/EEG_train.npy")
    noiseEEG_val=np.load("/kaggle/working/forpretrained/noiseEEG_val.npy")
    EEG_val=np.load("/kaggle/working/forpretrained/EEG_val.npy")

    learning_rate = trial.suggest_loguniform('learning_rate', 1e-3, 1e-1)

    optimizer = tf.optimizers.Adam(lr=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)

    

    ######################train############################
    @tf.function
    def train_step(model, noiseEEG_batch, EEG_batch, optimizer , denoise_network, batch_size, datanum):

        #本次训练参数初始化  parameter initialization in one step

        mse_grads = 0
        m_loss = 0


        with tf.GradientTape() as loss_tape:

            M_loss =  0
            for x in range(noiseEEG_batch.shape[0]):

                noiseeeg_batch,eeg_batch =  noiseEEG_batch[x] , EEG_batch[x]

                if denoise_network == 'fcNN':
                    noiseeeg_batch = tf.reshape(noiseeeg_batch, [1,datanum])
                else:
                    noiseeeg_batch = tf.reshape(noiseeeg_batch, [1,datanum,1])

                eeg_batch=tf.reshape(eeg_batch, [1,datanum,1])
                denoiseoutput = model(noiseeeg_batch)
                denoiseoutput = tf.reshape(denoiseoutput, [1,datanum,1])                          

                m_loss = denoise_loss_mse(denoiseoutput,eeg_batch)   
                M_loss += m_loss


            M_loss = M_loss / float(noiseEEG_batch.shape[0]) 

            # calculate gradient
            mse_grads = loss_tape.gradient(M_loss, model.trainable_variables)
            #bp
            optimizer.apply_gradients(zip(mse_grads, model.trainable_variables))

        return  M_loss,  mse_grads[0]  #每一条EEG的loss从此输出



    def train(model, noiseEEG,EEG, noiseEEG_val, EEG_val, epochs, batch_size,optimizer, denoise_network, result_location, foldername, train_num,prev_history,history):
        if prev_history ==False:
            # setup history variables and save history in a npy film
            history = {}
            history['grads'], history['loss']= {}, {}
            train_mse_history, val_mse_history = [],[]
            mse_grads_history = []
            val_mse_min = 100.0      # any number bigger than 1
        else:
            history=history
            train_mse_history, val_mse_history = history['loss']['train_mse'], history['loss']['val_mse']
            mse_grads_history = history['grads']['mse'] 
            hlv=history['loss']['val_mse']
            val_mse_min = float(hlv[-1]) 

        print('We are in the train fn')    
        # save history to tensorboard
        # current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
        train_log_dir = result_location +'/'+foldername +'/'+ train_num + '/train'
        val_log_dir = result_location +'/'+foldername +'/'+ train_num + '/test'
        train_summary_writer = tf.summary.create_file_writer(train_log_dir)
        val_summary_writer = tf.summary.create_file_writer(val_log_dir)

        batch_num = math.ceil(noiseEEG.shape[0]/batch_size)

        datanum = noiseEEG.shape[1]
        for epoch in range(epochs):
            start = time.time()

            # initialize  loss value for every epoch
            mse_grads , train_mse = 0, 0

            with tqdm(total=batch_num, position=0, leave=True) as pbar:

                for n_batch in range(batch_num):

                    #
                    if n_batch == batch_num:
                        noiseEEG_batch,EEG_batch =  noiseEEG[batch_size*n_batch :] , EEG[batch_size*n_batch :]
                    else:
                        noiseEEG_batch,EEG_batch =  noiseEEG[batch_size*n_batch : batch_size*(n_batch+1)] , EEG[batch_size*n_batch : batch_size*(n_batch+1)]

                    mse_loss_batch, mse_grads_batch = train_step(model, noiseEEG_batch,EEG_batch, optimizer, denoise_network, batch_size , datanum)

                    # convert variables to usable format
                    mse_grads_batch = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(mse_grads_batch)))).numpy()
                    mse_loss_batch = tf.reduce_mean(mse_loss_batch).numpy()

                    # store history 
                    train_mse += mse_loss_batch/float(batch_num)
                    mse_grads += mse_grads_batch/float(batch_num)

                    pbar.update()
                pbar.close()

            # store train history 
            mse_grads_history.append(mse_grads)
            train_mse_history.append(train_mse)

            with train_summary_writer.as_default():
                tf.summary.scalar('loss', train_mse, step=epoch)


            # calculate mse loss for validation set
            #denoiseoutput, val_mse, loss_rrmset = test_step(model, noiseEEG_val, EEG_val)
            denoiseoutput, val_mse = test_step(model, noiseEEG_val, EEG_val)

            #store validation history
            val_mse_history.append(val_mse) 

            with val_summary_writer.as_default():   # record validation loss to tensorboard
                tf.summary.scalar('loss', val_mse, step=epoch)



            if float(val_mse) < val_mse_min:  # if epoch_number > 0.8*all_epoch_number begin to save the best model  ## for SCNN or CCNN in EMG we should save the first or second model. 
                print('yes,smaller ', float(val_mse) ,val_mse_min)  
                val_mse_min = float(val_mse)
                saved_model = model

                history['grads']['mse'] = mse_grads_history
                history['loss']['train_mse'], history['loss']['val_mse']  = train_mse_history, val_mse_history
                # save dictionary to person_data.pkl file
                with open('/kaggle/working/forpretrained/history.pkl', 'wb') as fp:
                    pickle.dump(history, fp)


            print ('Epoch #: {}/{}, Time taken: {} secs,\n Grads: mse= {},\n Losses: train_mse= {}, val_mse={}'\
                         .format(epoch+1,epochs,time.time()-start , mse_grads,  train_mse, val_mse))


        #Generate after the final epoch
        clear_output(wait=True)

        #save history to dict
        history['grads']['mse'] = mse_grads_history
        history['loss']['train_mse'], history['loss']['val_mse']  = train_mse_history, val_mse_history

        return saved_model, history, val_mse


    #############################
    saved_model, history, val_mse_last = train(model, noiseEEG_train, EEG_train, noiseEEG_val, EEG_val, 
                      epochs, batch_size,optimizer, denoise_network, 
                      result_location, foldername , train_num = str(i),prev_history=prev_history,history=history)


    print(history)
    
#     # Simulate a long training process...wait 1 sec
#     time.sleep(1)

    trial.report(val_mse_last,trial.number)
    if trial.should_prune():
        raise optuna.exceptions.PrialPruned
    

    return val_mse_last

# Add stream handler of stdout to show the messages
optuna.logging.get_logger("optuna").addHandler(logging.StreamHandler(sys.stdout))
study_name = "mm-study"  # Unique identifier of the study.
storage_name = "sqlite:///{}.db".format(study_name)

study = optuna.create_study(direction='minimize',pruner=optuna.pruners.MedianPruner(),study_name=study_name, storage=storage_name, load_if_exists=True)

# study = optuna.create_study(direction='minimize',pruner=optuna.pruners.MedianPruner(),study_name=study_name, storage=storage_name)

study.optimize(objective, n_trials=50, callbacks=[lambda study, trial: gc.collect()])

In [ ]:
import tensorflow as tf
from keras import Sequential
from tensorflow import keras
import shutil
import os
import optuna
import pickle

shutil.copyfile("/kaggle/input/d/mayarmahmoud015/optuna/mm-study.db", "/kaggle/working/mm-study.db")

# Path to the trial number file
trial_num_file = "/kaggle/input/d/mayarmahmoud015/optuna/trial_num.pkl"

# Load the trial number
with open(trial_num_file, "rb") as f:
    trial_number = pickle.load(f)
    print('trial_number: ',trial_number)

In [ ]:
study = optuna.create_study(direction='minimize',pruner=optuna.pruners.MedianPruner(),study_name='mm-study', storage="sqlite:///{}.db".format('mm-study'), load_if_exists=True)

In [ ]:
from optuna.visualization import plot_contour
from optuna.visualization import plot_edf
from optuna.visualization import plot_intermediate_values
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances
from optuna.visualization import plot_rank
from optuna.visualization import plot_slice
from optuna.visualization import plot_timeline
plot_slice(study)

In [ ]:
plot_optimization_history(study)

# **end of optuna codes**

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np
# noiseEEG_test=np.load("/kaggle/working/forpretrained/noiseEEG_test.npy")
# EEG_test=np.load("/kaggle/working/forpretrained/EEG_test.npy")
# model =tf.keras.saving.load_model("/kaggle/working/forpretrained/my_model.keras")
# d = model(noiseEEG_test[5040].reshape(1,1024))
# l1,=plt.plot(EEG_test[5040])
# l2,=plt.plot(d[0])
# plt.legend([l1,l2], ['Ground_truth','denoised 3layers 10ep'])
# plt.show()

# l3,=plt.plot(noiseEEG_test[5040])
# plt.show()



In [ ]:
denoise = model(noiseEEG_test)
print(denoise.shape)

In [ ]:
#RRMSE
RRMSE_total = 0
RRMSE_arr = []
for j in range(10):
    print(j)
    for i in range(560):
        RRMSE = denoise_loss_rrmset(denoise[560*j+i], EEG_test[560*j+i])
        RRMSE_total = RRMSE_total + RRMSE
    RRMSE_total = RRMSE_total/560
    RRMSE_arr.append(RRMSE_total)
print(RRMSE_arr)
import numpy as np
import matplotlib.pyplot as plt
snr_values = np.linspace(-7, 2, 10)
plt.plot(snr_values, RRMSE_arr, marker='o', linestyle='-')
plt.xlabel('SNR (dB)')
plt.ylabel('RRMSE')
plt.title('LU-Net architecture')
plt.grid(True)
plt.show()

In [ ]:
#Correlation
import numpy as np
corr_total = 0
corr_arr = []
for j in range(10):
    print(j)
    for i in range(560):
        corr = np.corrcoef(denoise[560*j+i], EEG_test[560*j+i])[0,1]
        corr_total = corr_total + corr
    corr_total = corr_total/560
    corr_arr.append(corr_total)
print(corr_arr)
import numpy as np
import matplotlib.pyplot as plt
snr_values = np.linspace(-7, 2, 10)
plt.plot(snr_values, corr_arr, marker='o', linestyle='-')
plt.xlabel('SNR (dB)')
plt.ylabel('correlation')
plt.title('LU-Net architecture')
plt.grid(True)
plt.show(